# Module 01: LangGraph Fundamentals - Follow Along

**Purpose:** Run all examples from the theory document  
**How to use:** Execute each cell in order to see LangGraph in action  
**After this:** Move to `module-01-practice.ipynb` for exercises

---

This notebook contains all working code examples from Module 01 theory document.
Run each cell and observe the outputs to understand how LangGraph works.


## Setup


In [ ]:
# Install dependencies
%pip install -q -U langgraph langchain langchain-openai python-dotenv

import os
from dotenv import load_dotenv
load_dotenv()

print('✅ Environment ready!')

## Example 1: Understanding State

State is the shared data structure that flows through your graph.


In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# Define a state schema
class SimpleState(TypedDict):
    messages: Annotated[list, add_messages]  # Messages with special reducer
    user_name: str

# Example state
example_state = {
    'messages': [],
    'user_name': 'Alice'
}

print(f"State schema defined: {SimpleState.__annotations__}")
print(f"Example state: {example_state}")

## Example 2: Creating Nodes

Nodes are functions that process state and return updates.


In [ ]:
# Simple node example
def my_node(state: SimpleState) -> SimpleState:
    """A node that processes state."""
    result = f"Hello, {state['user_name']}!"
    return {"user_name": state['user_name'].upper()}  # Return update

# Test the node
test_state = {'messages': [], 'user_name': 'alice'}
update = my_node(test_state)
print(f"Input: {test_state}")
print(f"Update returned: {update}")
print(f"Note: Node returns ONLY updates, not full state")

## Example 3: Building Your First Graph

This is the complete chatbot example from the theory document.


In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# State schema for chatbot
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]
    user_name: str

# Node 1: Extract user name
def extract_name_node(state: ChatState) -> ChatState:
    """Extract user name from first message if not already known."""
    if state.get('user_name'):
        return {}  # No update needed
    
    first_message = state['messages'][0].content if state['messages'] else ""
    if "my name is" in first_message.lower():
        name = first_message.lower().split("my name is")[1].strip().split()[0]
        return {"user_name": name.capitalize()}
    
    return {}

# Node 2: Chatbot (simplified - no actual LLM call)
def chatbot_node(state: ChatState) -> ChatState:
    """Simulate chatbot response."""
    user_name = state.get('user_name', 'there')
    response = AIMessage(content=f"Hello {user_name}! How can I help you today?")
    return {"messages": [response]}

# Build the graph
workflow = StateGraph(ChatState)
workflow.add_node("extract_name", extract_name_node)
workflow.add_node("chatbot", chatbot_node)

# Define edges
workflow.add_edge(START, "extract_name")
workflow.add_edge("extract_name", "chatbot")
workflow.add_edge("chatbot", END)

# Compile
app = workflow.compile()

print("✅ Graph built successfully!")
print(f"Nodes: {list(app.get_graph().nodes.keys())}")

## Example 4: Running the Graph


In [ ]:
# First interaction
initial_state = {
    "messages": [HumanMessage(content="Hi, my name is Alice")],
    "user_name": ""
}

result = app.invoke(initial_state)
print("First interaction:")
print(f"  User said: {result['messages'][0].content}")
print(f"  Bot replied: {result['messages'][-1].content}")
print(f"  User name extracted: {result['user_name']}")

In [ ]:
# Follow-up (the graph remembers the name)
follow_up_state = {
    "messages": result['messages'] + [HumanMessage(content="What's my name?")],
    "user_name": result['user_name']
}

result2 = app.invoke(follow_up_state)
print("\nFollow-up interaction:")
print(f"  User asked: {follow_up_state['messages'][-1].content}")
print(f"  Bot knows: {result2['user_name']}")
print(f"  ✅ State persisted across invocations!")

## Example 5: Conditional Routing

Demonstrate dynamic routing based on state.


In [ ]:
class RouterState(TypedDict):
    score: float
    path: str
    result: str

def classifier(state: RouterState):
    """Classify based on score."""
    return {'score': state.get('score', 0.5)}

def high_confidence_handler(state: RouterState):
    return {'result': f"High confidence path (score: {state['score']})"}

def low_confidence_handler(state: RouterState):
    return {'result': f"Low confidence path (score: {state['score']})"}

# Router function
def route_logic(state: RouterState) -> str:
    if state['score'] > 0.8:
        return "high_confidence"
    return "low_confidence"

# Build graph with conditional edges
workflow = StateGraph(RouterState)
workflow.add_node("classifier", classifier)
workflow.add_node("high_confidence", high_confidence_handler)
workflow.add_node("low_confidence", low_confidence_handler)

workflow.add_edge(START, "classifier")
workflow.add_conditional_edges(
    "classifier",
    route_logic,
    {"high_confidence": "high_confidence", "low_confidence": "low_confidence"}
)
workflow.add_edge("high_confidence", END)
workflow.add_edge("low_confidence", END)

router_app = workflow.compile()

# Test with different scores
for score in [0.9, 0.5]:
    result = router_app.invoke({'score': score, 'path': '', 'result': ''})
    print(f"Score {score}: {result['result']}")

## Example 6: Using Reducers

Custom reducers define how state updates are merged.


In [ ]:
# Custom reducer example
def accumulate_scores(existing: list, new: list) -> list:
    """Accumulate scores instead of replacing."""
    return existing + new

class ScoreState(TypedDict):
    scores: Annotated[list, accumulate_scores]

def score_node_1(state: ScoreState):
    return {"scores": [0.7, 0.8]}

def score_node_2(state: ScoreState):
    return {"scores": [0.9]}  # Will be appended, not replaced

# Build graph
workflow = StateGraph(ScoreState)
workflow.add_node("node1", score_node_1)
workflow.add_node("node2", score_node_2)
workflow.add_edge(START, "node1")
workflow.add_edge("node1", "node2")
workflow.add_edge("node2", END)

score_app = workflow.compile()
result = score_app.invoke({'scores': []})

print(f"Scores accumulated: {result['scores']}")
print(f"Without reducer, only [0.9] would remain!")

## Example 7: Streaming Execution

See intermediate results as the graph executes.


In [ ]:
class StreamState(TypedDict):
    count: int

def step1(state: StreamState):
    return {'count': state.get('count', 0) + 1}

def step2(state: StreamState):
    return {'count': state['count'] + 1}

def step3(state: StreamState):
    return {'count': state['count'] + 1}

workflow = StateGraph(StreamState)
workflow.add_node("step1", step1)
workflow.add_node("step2", step2)
workflow.add_node("step3", step3)
workflow.add_edge(START, "step1")
workflow.add_edge("step1", "step2")
workflow.add_edge("step2", "step3")
workflow.add_edge("step3", END)

stream_app = workflow.compile()

print("Regular invoke (blocking):")
result = stream_app.invoke({'count': 0})
print(f"  Final count: {result['count']}")

print("\nStreaming (see each step):")
for i, chunk in enumerate(stream_app.stream({'count': 0})):
    print(f"  Step {i+1}: {chunk}")

## Example 8: Error Handling Pattern


In [ ]:
class ErrorState(TypedDict):
    value: int
    status: str
    error: str

def risky_operation(state: ErrorState):
    """Operation that might fail."""
    try:
        if state['value'] < 0:
            raise ValueError("Negative values not allowed")
        result = 100 / state['value']  # Division by zero possible
        return {"status": "success", "value": int(result)}
    except Exception as e:
        return {"status": "error", "error": str(e)}

def route_on_status(state: ErrorState) -> str:
    return "handle_error" if state['status'] == "error" else "success"

def handle_error(state: ErrorState):
    return {"value": 0, "error": f"Handled: {state.get('error', 'unknown')}"}

def handle_success(state: ErrorState):
    return {"status": "completed"}

# Build graph
workflow = StateGraph(ErrorState)
workflow.add_node("risky", risky_operation)
workflow.add_node("handle_error", handle_error)
workflow.add_node("success", handle_success)

workflow.add_edge(START, "risky")
workflow.add_conditional_edges(
    "risky",
    route_on_status,
    {"handle_error": "handle_error", "success": "success"}
)
workflow.add_edge("handle_error", END)
workflow.add_edge("success", END)

error_app = workflow.compile()

# Test success case
result = error_app.invoke({'value': 10, 'status': '', 'error': ''})
print(f"Success case (value=10): status={result['status']}")

# Test error case
result = error_app.invoke({'value': 0, 'status': '', 'error': ''})
print(f"Error case (value=0): {result['error']}")

## Summary

You've seen:
- ✅ State schemas with TypedDict
- ✅ Node functions that return updates
- ✅ Building graphs with StateGraph
- ✅ START and END constants
- ✅ Normal and conditional edges
- ✅ Custom reducers
- ✅ Streaming execution
- ✅ Error handling patterns

**Next Steps:**
1. Review the theory document for detailed explanations
2. Try `module-01-practice.ipynb` for hands-on exercises
3. Build your own graphs!
